# Customer Churn Prediction: from Python to Modeler and back to Python


Adapted from original by Heba El-Shimy https://raw.githubusercontent.com/IBM/customer-churn-prediction/master/notebooks/customer-churn-prediction.ipynb 

-------------------

- This article: https://medium.com/@markryan_69718/watson-studio-desktop-first-impressions-5a85309597d0 describes adapting the original Python-based churn solution described here: https://developer.ibm.com/patterns/predict-customer-churn-using-watson-studio-and-jupyter-notebooks/
- the article describes a simple Modeler flow https://github.com/ryanmark1867/shared_ml/blob/master/churn%20flow%20Feb%202019.str?raw=true that implements a subset of the original Python-based churn solution
- this notebook takes things full circle by attempting to implement the Modeler flow "note for note" back into a much simplified Python notebook that captures nuances of the Modeler flow

This notebook shows screenshots of the Modeler flow followed by the Python that attempts to implement the same function.


# Load Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
import sklearn.feature_selection
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn import metrics
from scipy import stats
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
import pixiedust

testproportion = 0.3


Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


# The Dataset

The same dataset used in this notebook was used in the Modeler flow described above.

From a telecommunications company. It includes information about:  
- Customers who left within the last month – the column is called Churn

- Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies

- Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges

- Demographic info about customers – gender, age range, and if they have partners and dependents

Link for getting the dataset: [https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Telco-Customer-Churn.csv](https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Telco-Customer-Churn.csv)



# Ingest dataset
- the first step in the Modeler flow is to ingest the dataset
- the code below brings the dataset into a Pandas dataframe
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_ingest_data.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>


### 2. Loading Our Dataset

Click on the cell below to highlight it.

Then go to the `Files` section to the right of this notebook and click `Insert to code` for the data you have uploaded. Choose `Insert pandas DataFrame`.

In [2]:
# The code was removed by Watson Studio for sharing.

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
customer_data = df_data_1

In [4]:
!pwd

/home/dsxuser/work


# Select subset of columns from original dataset
- the next node of the Modeler flow selects a subset of the columns from the original dataset

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_filter_selected.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [5]:
# define the columns that we are going to keep
retain_columns = ['MonthlyCharges','TotalCharges','InternetService','PaymentMethod','OnlineSecurity','Churn','Contract','tenure']

In [6]:
# select a subset of columns from the original dataset
customer_data = customer_data[['MonthlyCharges','TotalCharges','InternetService','PaymentMethod','OnlineSecurity','Churn','Contract','tenure']]
customer_data.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,29.85,29.85,DSL,Electronic check,No,No,Month-to-month,1
1,56.95,1889.50,DSL,Mailed check,Yes,No,One year,34
2,53.85,108.15,DSL,Mailed check,Yes,Yes,Month-to-month,2
3,42.30,1840.75,DSL,Bank transfer (automatic),Yes,No,One year,45
4,70.70,151.65,Fiber optic,Electronic check,No,Yes,Month-to-month,2


# Fill empty values in TotalCharges column
- in the Modeler flow we only fill empty values in the TotalCharges column
- do the same in Python

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_filler.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [7]:
# check the number of empty values in TotalCharges before and after filling the missing values
print("empty values in TotalCharges before filling:",customer_data['TotalCharges'].isnull().sum())
customer_data['TotalCharges'].fillna(value=0, inplace=True)
print("empty values in TotalCharges after filling:",customer_data['TotalCharges'].isnull().sum())

empty values in TotalCharges before filling: 11
empty values in TotalCharges after filling: 0


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


# Prep data
The next step in the Modeler flow incorporates many preparation steps, including:
1. replacing categorical tokens with numerical IDs
2. scaling continuous values
3. replace outliers (values that are beyond a threshold) with the threshold

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [8]:
# identify the subsets of columns that are categorical and continuous
categorical_columns = ['InternetService', 'PaymentMethod', 'OnlineSecurity', 'Churn', 'Contract']
continuous_columns = ['MonthlyCharges', 'TotalCharges','tenure']

In [9]:
# input dataframe and list of colums to be zscore scaled' return dataframe with those columns scaled
def scale_columns(df,col_list):
    for col in col_list:
        print("col is",col)
        df[col] = df[col].apply(zscore)
    return df

In [10]:
# input dataframe and list of columns to be encoded; return dataframe with those columns encoded
def encode_columns(df,col_list):
    for col in col_list:
        print("col is",col)
        le = LabelEncoder()
        le.fit(df[col].tolist())
        df[col] = le.transform(df[col])
    return(df)
 

In [11]:
# replace tokens in categorical columns with numeric IDs
customer_data = encode_columns(customer_data,categorical_columns)
customer_data.head()

col is InternetService
col is PaymentMethod


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


col is OnlineSecurity
col is Churn
col is Contract


,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,29.85,29.85,0,2,0,0,0,1
1,56.95,1889.50,0,3,2,0,1,34
2,53.85,108.15,0,3,2,1,0,2
3,42.30,1840.75,0,0,2,0,1,45
4,70.70,151.65,1,2,0,1,0,2


In [12]:
# print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())

min MonthlyCharges 18.25
max TotalCharges 8684.8
min TotalCharges 0.0
max tenure 72
min tenure 0


# Prep data 2 - scale continuous values


<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep_scaling.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [13]:
# scale continuous columns using zscore
customer_data[continuous_columns] = customer_data[continuous_columns].apply(zscore)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


# Prep data 3: replace outliers
- replace outliers (values that are beyond a threshold) with the threshold

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep_outliers_replace.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [14]:
def sd_max(x,sd,multiplier):
    if x > multiplier*sd:
        return multiplier*sd
    else:
        return x

def replace_outliers(df,multiplier):
    for col in continuous_columns:
        sd = df.loc[:,col].std()
        print("sd",sd)
        df[col] = df[col].apply(sd_max,args=(sd,multiplier))
    return df
        
        

In [15]:
# replace outliers that are more than a boundary with the boundary value
customer_data = replace_outliers(customer_data,3.0)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


sd 1.0000710000355904
sd 1.0000710000355884
sd 1.0000710000355943


In [16]:
print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())

max MonthlyCharges 1.79435215026
min MonthlyCharges -1.54585982007
max TotalCharges 2.82580557787
min TotalCharges -1.00577983371
max tenure 1.61370124044
min tenure -1.3181649474


In [17]:
customer_data.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,-1.160323,-0.992611,0,2,0,0,0,-1.277445
1,-0.259629,-0.172165,0,3,2,0,1,0.066327
2,-0.362660,-0.958066,0,3,2,1,0,-1.236724
3,-0.746535,-0.193672,0,0,2,0,1,0.514251
4,0.197365,-0.938874,1,2,0,1,0,-1.236724


In [18]:
print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())


max MonthlyCharges 1.79435215026
min MonthlyCharges -1.54585982007
max TotalCharges 2.82580557787
min TotalCharges -1.00577983371
max tenure 1.61370124044
min tenure -1.3181649474


# Data in the Modeler flow post Auto Data Prep
- values in categorical columns have been replaced with numerical IDs
- the continuous columns have been scaled with a zscore transformation and outliers have been replaced
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_preview_auto_data_prep.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Split dataset into test and train
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_partition.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [19]:
# define label
y_le = customer_data['Churn']
# define input values
X_selected = customer_data.drop(['Churn'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_le,\
                                                    test_size=testproportion, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4930, 7) (4930,)
(2113, 7) (2113,)


In [20]:
X_test.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Contract,tenure
185,-1.328164,-0.994838,0,2,0,0,-1.277445
2715,-1.313208,-0.566163,2,0,1,0,0.351370
3825,-1.509300,-0.550611,2,3,1,2,0.799294
1807,0.385148,-0.972096,1,2,0,0,-1.277445
132,-0.472339,0.432521,0,0,0,2,1.410099


# Train Support Vector Machine
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_SVM_train.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [21]:
# fit SVM using training data
clf_svc = svm.SVC(random_state=42)
clf_svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

# Train Logistic Regression
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_logistic_regression_train.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [22]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
model = clf_lr.fit(X_train, y_train)
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Evaluate Support Vector Machine
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_SVM_results2.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

Results from Modeler flow SVM:

Results for output field Churn_transformed
Comparing $S-Churn_transformed with Churn_transformed
'Partition'            1_Training            2_Testing         
Correct                     3,859   78.56%       1,722   80.81%
Wrong                       1,053   21.44%         409   19.19%
Total                       4,912                2,131    

In [23]:
# Get model confidence of predictions
y_score_svc = clf_svc.decision_function(X_test)
y_score_svc

array([-0.06117747, -1.01810821, -1.01249794, ..., -1.01551142,
       -0.02449712, -1.04244246])

In [24]:
# Get accuracy score
y_pred_svc = clf_svc.predict(X_test)
acc_svc = accuracy_score(y_test, y_pred_svc)
print(acc_svc)

0.795551348793


In [25]:
# Get Precision vs. Recall score
average_precision_svc = average_precision_score(y_test, y_score_svc)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision_svc))

Average precision-recall score: 0.64


# Evaluate Logistic Regression
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_logistic_regression_results.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

Results for output field Churn_transformed
Comparing $L-Churn_transformed with Churn_transformed
'Partition'            1_Training            2_Testing         
Correct                     3,891   79.21%       1,725   80.95%
Wrong                       1,021   20.79%         406   19.05%
Total                       4,912                2,131         
Performance Evaluation
'Partition' = 1_Training                 
0                                   0.857
1                                    0.14
'Partition' = 2_Testing                 
0                                  0.943
1                                  0.129

In [26]:
y_score_lr = clf_lr.decision_function(X_test)
y_score_lr

array([-0.55997081, -2.6605672 , -5.07462064, ..., -4.51331337,
        0.30978154, -0.71314255])

In [27]:
y_pred_lr = clf_lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(acc_lr)

0.793658305726


In [28]:
average_precision_lr = average_precision_score(y_test, y_score_lr)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision_lr))

Average precision-recall score: 0.64
